In [ ]:
import streamlit as st
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image
import pandas as pd

# Load the model
plant_model = tf.keras.models.load_model('custom_plant_model.h5')
disease_classes = ['Apple_healthy', 'Tomato_early_blight', 'Potato_late_blight']  # Full list of 38 classes

st.title("Custom Plant Health Analyzer")
st.sidebar.header("Settings")
conf_threshold = st.sidebar.slider("Confidence Threshold (%)", 0, 100, 50)

uploaded_img = st.file_uploader("Choose a leaf image", type=["jpg", "png", "jpeg"])

if uploaded_img is not None:
    progress = st.progress(0)
    img = image.load_img(uploaded_img, target_size=(224, 224))
    st.image(img, caption="Your Uploaded Image", use_column_width=True)

    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    progress.progress(50)  # Simulate loading
    prediction_scores = plant_model.predict(img_array)[0]
    progress.progress(100)

    # Filter and display results in a table
    filtered_results = [(disease_classes[i], score * 100) for i, score in enumerate(prediction_scores) if score * 100 >= conf_threshold]
    if filtered_results:
        df = pd.DataFrame(filtered_results, columns=["Disease", "Confidence (%)"])
        st.table(df.sort_values(by="Confidence (%)", ascending=False))
    else:
        st.write("No predictions above the confidence threshold.")
